In [1]:
import spacy
import csv
import json
import re
from subprocess import Popen
from subprocess import PIPE

In [2]:
def extractContent(data):
    lstContent = list()
    for key in data:
        if type(data[key]) == dict:
            lstContent += extractContent(data[key])
        elif key.lower() == 'contenido':
            lstContent.append(str(data[key]).replace('-',''))
    return lstContent

def identifyEntity(sentence):
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(".", "")
    lstEntities = []
    for word in sentence.split():
        if word in dictEntities.values():
            if word not in lstEntities:
                lstEntities.append(word)
    return lstEntities

In [3]:
nlp = spacy.load('es_core_news_lg')
with open('../148.json') as file:
    data = json.load(file)

In [4]:
dictEntities = {}
with open('../listRelacion.csv') as file: 
    reader = csv.reader(file)
    for row in reader:
        if 'Entidad' == row[0]: continue
        key = row[0]
        dictEntities[key] = row[1:][0]

In [5]:
listEntities_sort = sorted(dictEntities.keys(), key = lambda palabra: len(palabra), reverse=True)

In [6]:
lstContent = extractContent(data) 

In [7]:
lstSentiments = [['Entidad', 'Párrafo', 'Puntución Positiva', 'Puntución Negativa',
                 'Palabras que dan la carga']]
for paragraph in lstContent:
    paragraph = re.sub(r'\r\n','', str(paragraph))
    paragraph = paragraph.replace("\n"," ") 
    for entity in listEntities_sort:
        paragraph = paragraph.replace(entity, dictEntities[entity])
    identifySentiment = 0
    for entity in identifyEntity(paragraph):
        if not identifySentiment:
            identifySentiment = 1
            sentStrength = Popen(["java", "-jar", "./java/SentStrength/SentiStrengthCom.jar", "sentidata", 
                              "./java/SentStrength_Data/Español/", "text", paragraph, "sentenceCombineTot", 
                              "paragraphCombineTot", "explain", "trinary"], stdout = PIPE)
            result = sentStrength.communicate()[0].decode('utf-8')
            polarity = result.split()[0:2]
            wordsPolarity = re.findall(r'(\w+\[\-?[0-9]+\])', result)
        lstSentiments.append([entity, paragraph, polarity[0], polarity[1], wordsPolarity])

In [9]:
with open('paragraphPolarity.csv', 'w', encoding = 'utf_8_sig') as file:
    writer = csv.writer(file)
    writer.writerows(lstSentiments)